In [ ]:
!pip install emoji emojis openpyxl

In [ ]:
!spacy download pt_core_news_sm

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import emojis
import re
import regex

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import spacy

spc = spacy.load("pt_core_news_sm")

In [ ]:
default_dir = "/data/"
raw_data = default_dir + 'raw_data/raw_tweet_collect.csv'
emoji_content_data = default_dir + 'raw_data/dataset_with_emojis.csv'


In [ ]:
data = pd.read_csv()
data.shape

(811440, 22)

In [ ]:
data.head(1)

,Unnamed: 0,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,0,https://twitter.com/Divergente1977/status/1365...,2021-02-27 23:59:50+00:00,DESRESPEITO COM MAIS DE 254 Mil brasileiros mo...,DESRESPEITO COM MAIS DE 254 Mil brasileiros mo...,1365813925560909824,"{'username': 'Divergente1977', 'displayname': ...",['https://twitter.com/veramagalhaes/status/136...,['https://t.co/c7a59PBfhr'],0,3,4,0,1365813925560909824,pt,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,{'url': 'https://twitter.com/veramagalhaes/sta...,NaN


## Limpeza dos textos de tweets

In [ ]:
def clean_text(texto):
    '''
    Função para converter todas as letras para sua forma minúscula, selecionar apenas as letras,
    remover stopwords e lematizar o texto. 
    '''

    def remove_tweet_ruido(tweets_text):
        '''
        Tweet cleansing.
        '''

        clean_text = re.sub(r'RT+', '', tweets_text) 
        clean_text = re.sub(r'@\S+', '', clean_text)  
        clean_text = re.sub(r'http\S+', '', clean_text) 
        clean_text = clean_text.replace("\n", " ")

        return clean_text
    
    ### Transforme as letras para minúscula ###
    minusculas = texto.lower()

    texto = remove_tweet_ruido(minusculas)
    
    ### Selecione apenas as letras do texto ##
    letras = re.findall(r'\b[A-zÀ-úü]+\b', texto)
    
    ### Removendo as stopwords ###
    stops = set(stopwords.words('portuguese')) 
    # Retire as stopwords de letras
    palavras_sem_stopwords = [palavra for palavra in letras if palavra not in stops]
    # Junte as palavras sem stopwords 
    palavras_importantes = ' '.join(palavras_sem_stopwords) 
    
    ### Lematização ###
    spc_letras = spc(palavras_importantes)
    # Lematize o texto 
    lemmas = [token.lemma_ if token.pos_ == 'VERB' else str(token) for token in spc_letras]
    # Junte os lemmas 
    texto_limpo = " ".join(lemmas)
    
    return texto_limpo

## Emoji classification - Positivo/Negativo/Neutro

Estamos considerando 118 emojis seguindo https://blog.emojipedia.org/emoji-use-in-the-new-normal/#fn1


In [ ]:
emojidict = {
'😀':"Positivo",'😁':"Positivo",'😂':"Positivo",'😃':"Positivo",'😄':"Positivo",'😅':"Positivo",'😆':"Positivo",'😉':"Positivo",'😊':"Positivo",
'😋':"Positivo",'😎':"Positivo",'😍':"Positivo",'😘':"Positivo",'😗':"Positivo",'😙':"Positivo",'😚':"Positivo",'☺️':"Positivo",'🤗':"Positivo",
'😇':"Positivo",'😏':"Positivo",'😌':"Positivo",'😛':"Positivo",'😜':"Positivo",'😝':"Positivo",'🤑':"Positivo",'😈':"Positivo",'😸':"Positivo",
'😹':"Positivo",'😺':"Positivo",'😻':"Positivo",'😼':"Positivo",'😽':"Positivo",'🤠':"Positivo",'🤣':"Positivo",'🤤':"Positivo",'🤩':"Positivo",
'🤪':"Positivo",'🥳':"Positivo",'🥰':"Positivo",'❤️':"Positivo",'👍':"Positivo",'🤔':"Negativo",'😐':"Negativo",'😑':"Negativo",'😶':"Negativo",
'🙄':"Negativo",'😣':"Negativo",'😥':"Negativo",'😮':"Negativo",'🤐':"Negativo",'😯':"Negativo",'😪':"Negativo",'😫':"Negativo",'😴':"Negativo",
'☹️':"Negativo",'🙁':"Negativo",'😒':"Negativo",'😓':"Negativo",'😔':"Negativo",'😕':"Negativo",'😖':"Negativo",'🙃':"Negativo",'😷':"Negativo",
'🤒':"Negativo",'🤕':"Negativo",'😲':"Negativo",'😞':"Negativo",'😟':"Negativo",'😤':"Negativo",'😢':"Negativo",'😭':"Negativo",'😦':"Negativo",
'😧':"Negativo",'😨':"Negativo",'😩':"Negativo",'😬':"Negativo",'😰':"Negativo",'😱':"Negativo",'😳':"Negativo",'😵':"Negativo",'😡':"Negativo",
'😠':"Negativo",'👿':"Negativo",'👹':"Negativo",'💀':"Negativo",'☠️':"Negativo",'😾':"Negativo",'😿':"Negativo",'🙀':"Negativo",'🤢':"Negativo",
'🤥':"Negativo",'🤧':"Negativo",'🤨':"Negativo",'🤬':"Negativo",'🤮':"Negativo",'🦠':"Negativo",'🤓':"Neutro",'🗣️':"Neutro",'👤':"Neutro",
'👥':"Neutro",'👺':"Neutro",'👻':"Neutro",'👽':"Neutro",'👾':"Neutro",'🤖':"Neutro",'💩':"Neutro",'🤡':"Neutro",'🤫':"Neutro",
'🤭':"Neutro",'🤯':"Neutro",'🧐':"Neutro",'🥴':"Neutro",'🥵':"Neutro",'🥶':"Neutro",'🥺':"Neutro",'🥱':"Neutro",'🤔':"Neutro",
'🤭':"Neutro",'🙏':"Neutro"
}

In [ ]:
def label_by_emoji(text):
  curr_emojis = [c for c in text if c in emojidict]
  if not curr_emojis:
    return None
  max_emoji = max(set(curr_emojis), key = curr_emojis.count)
  return emojidict[max_emoji]
  


In [ ]:
select_df = data[['date','renderedContent']].rename(columns={'renderedContent':'tweet_text'})
select_df

In [ ]:
select_df['sentiment'] = select_df['tweet_text'].apply(label_by_emoji)

In [ ]:
select_df.shape

(811440, 3)

In [ ]:
data_unlabelled = select_df.loc[select_df['sentiment'].isnull(), ['date','tweet_text']][:600000]
data_unlabelled['clean_text'] = data_unlabelled['tweet_text'].apply(clean_text)
data_unlabelled.to_csv(default_dir+'raw_data/dataset_unlabelled.csv', index=False)
print(data_unlabelled.shape)
data_unlabelled.head()

In [ ]:
data_labeled = select_df.loc[select_df['sentiment'].notnull(), ['date','tweet_text', 'sentiment']]
data_labeled['clean_text'] = data_labeled['tweet_text'].apply(clean_text)
data_labeled.shape

(81998, 4)

In [ ]:
## salvar dataset raw dos selecionados com emoji
# data_labeled_raw = data_labeled[['date','tweet_text', 'clean_text']]
# data_labeled_raw.head()
# data_labeled_raw.to_csv(default_dir+'raw_data/dataset_with_emojis.csv', index=False)

In [ ]:
emotion_dispersao = data_labeled['sentiment'].value_counts()
plot = emotion_dispersao.plot.pie(y='mass', figsize=(10, 15))

In [ ]:
data_labeled.head()

,date,tweet_text,sentiment,clean_text
13,2021-02-27 23:43:12+00:00,Jornal falando das medidas restritivas em uma ...,Neutro,jornal falar medidas restritivas capital acade...
42,2021-02-27 22:22:07+00:00,Os nossos vizinhos com medidas restritivas:\n\...,Negativo,vizinhos medidas restritivas flávio dino
43,2021-02-27 22:21:02+00:00,@Su_eline Enquanto as pessoas não acordarem pa...,Negativo,enquanto pessoas acordar gravidade situação ac...
49,2021-02-27 22:04:55+00:00,2de2\nE demorou muito para adotar medidas REST...,Negativo,demorar adotar medidas restritivas severas fic...
54,2021-02-27 22:00:52+00:00,"@JanainaDoBrasil 🤣🤣🤣😂😅\nQue lógica bisonha!\n""...",Positivo,lógica bisonha vamos esperar todas vagas ser o...


In [ ]:
data_labeled.to_csv(default_dir+'labeled_data/dataset_label_pos_neg_neutro.csv', index=False)

In [ ]:
data_pos_neg = data_labeled.loc[data_labeled['sentiment'] !='Neutro']
data_pos_neg.to_csv(default_dir+'labeled_data/dataset_label_pos_neg.csv', index=False)
